In [2]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
pd.set_option("display.precision", 1)

In [7]:
data = np.load(r'C:\Users\varad\OneDrive\Desktop\datasets\AndreNg\unsupervised learning\ContestBased\model_data.npy', allow_pickle=True)


In [8]:
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = data

num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
print(f"Number of training vectors: {len(item_train)}")

Number of training vectors: 50884


In [10]:
# scale training data
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


In [11]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (40707, 17)
movie/item test data shape: (10177, 17)


In [15]:
num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([ 
    tf.keras.layers.Dense(units=256,activation='relu'),
    tf.keras.layers.Dense(units=128,activation='relu'),
    tf.keras.layers.Dense(units=num_outputs,activation='linear')

])

item_NN = tf.keras.models.Sequential([   
    tf.keras.layers.Dense(units=256,activation='relu'),
    tf.keras.layers.Dense(units=128,activation='relu'),
    tf.keras.layers.Dense(units=num_outputs,activation='linear')

])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features,))
vu = user_NN(input_user)
from tensorflow.keras.layers import Lambda

vu = Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vu)


# create the item input and point to the base network
input_item = input_item = tf.keras.layers.Input(shape=(num_item_features,))

vm = item_NN(input_item)
vm = Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vm)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)    │ (None, 14)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_8 (InputLayer)    │ (None, 16)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ sequential_6 (Sequential)     │ (None, 32)                │          40,864 │ input_layer_6[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ sequential_7 (Sequential)     │ (None, 32)                │          41,376 │ input_layer_8[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lambda_1 (Lambda)             │ (None, 32)                │               0 │ sequential_6[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lambda_2 (Lambda)             │ (None, 32)                │               0 │ sequential_7[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dot (Dot)                     │ (None, 1)                 │               0 │ lambda_1[0][0],            │
│                               │                           │                 │ lambda_2[0][0]             │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 82,240 (321.25 KB)

 Trainable params: 82,240 (321.25 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [17]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Epoch 1/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.1293
Epoch 2/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1142
Epoch 3/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1102
Epoch 4/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1070
Epoch 5/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1028
Epoch 6/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0997
Epoch 7/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0975
Epoch 8/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0957
Epoch 9/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0939
Epoch 10/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0923
Epoch 11/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0907
Epoch 12/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0893
Epoch 13/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0881
Epoch 14/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0867
Epoch 15/30
1273/1273 ━━━━━━━

In [18]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

319/319 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0874


0.08410988748073578

prediction for new user

In [19]:
new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 0.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_horror = 0.0
new_mystery = 0.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

In [22]:
user_vec

array([[5.e+03, 3.e+00, 0.e+00, 0.e+00, 5.e+00, 0.e+00, 0.e+00, 0.e+00,
        0.e+00, 0.e+00, 0.e+00, 5.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
        0.e+00]])

Finding Similar Items

In [25]:
def sq_dist(a,b):

    d=0
    for i in range(a.shape[0]):
        d+=(a[i]-b[i])**2
    
    return d

In [26]:
a1 = np.array([1.0, 2.0, 3.0]); b1 = np.array([1.0, 2.0, 3.0])
a2 = np.array([1.1, 2.1, 3.1]); b2 = np.array([1.0, 2.0, 3.0])
a3 = np.array([0, 1, 0]);       b3 = np.array([1, 0, 0])
print(f"squared distance between a1 and b1: {sq_dist(a1, b1):0.3f}")
print(f"squared distance between a2 and b2: {sq_dist(a2, b2):0.3f}")
print(f"squared distance between a3 and b3: {sq_dist(a3, b3):0.3f}")

squared distance between a1 and b1: 0.000
squared distance between a2 and b2: 0.030
squared distance between a3 and b3: 2.000


In [32]:
input_item_m = tf.keras.layers.Input(shape=(num_item_features,))

vm_m = item_NN(input_item_m)    
from tensorflow.keras.layers import Lambda

#L2 normalization using Lambda layer
vm_m = Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vm_m)
                    
model_m = tf.keras.Model(input_item_m, vm_m)                                
model_m.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)          │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_7 (Sequential)            │ (None, 32)                  │          41,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_3 (Lambda)                    │ (None, 32)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 41,376 (161.62 KB)

 Trainable params: 41,376 (161.62 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])
print(f"size of all predicted movie feature vectors: {vms.shape}")

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
size of all predicted movie feature vectors: (847, 32)


In [34]:
vms

array([[-0.1619505 ,  0.2306573 ,  0.12416455, ..., -0.07036273,
         0.12491334, -0.00221958],
       [-0.11072747,  0.1920807 ,  0.08186378, ..., -0.09391677,
         0.02080891, -0.00808877],
       [-0.13438553,  0.0881383 , -0.02893417, ..., -0.01687917,
         0.13168283, -0.17871863],
       ...,
       [ 0.22425343, -0.09601087, -0.08462178, ...,  0.03914402,
        -0.2007732 , -0.05306713],
       [ 0.4462453 , -0.05720286,  0.05996634, ...,  0.03659994,
        -0.01098295, -0.19764479],
       [-0.05531217,  0.01364997, -0.15249486, ...,  0.13020812,
        -0.09993377, -0.0879766 ]], dtype=float32)

In [35]:
count = 50  # number of movies to display
dim = len(vms)
dist = np.zeros((dim,dim))

for i in range(dim):
    for j in range(dim):
        dist[i,j] = sq_dist(vms[i, :], vms[j, :])
        
m_dist = ma.masked_array(dist, mask=np.identity(dist.shape[0]))  # mask the diagonal

disp = [["movie1", "genres", "movie2", "genres"]]
for i in range(count):
    min_idx = np.argmin(m_dist[i])
    movie1_id = int(item_vecs[i,0])
    movie2_id = int(item_vecs[min_idx,0])
    disp.append( [movie_dict[movie1_id]['title'], movie_dict[movie1_id]['genres'],
                  movie_dict[movie2_id]['title'], movie_dict[movie1_id]['genres']]
               )
table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
table

movie1,genres,movie2,genres
Save the Last Dance (2001),Drama|Romance,Mona Lisa Smile (2003),Drama|Romance
"Wedding Planner, The (2001)",Comedy|Romance,Mr. Deeds (2002),Comedy|Romance
Hannibal (2001),Horror|Thriller,Final Destination 2 (2003),Horror|Thriller
Saving Silverman (Evil Woman) (2001),Comedy|Romance,"Wedding Planner, The (2001)",Comedy|Romance
Down to Earth (2001),Comedy|Fantasy|Romance,Bewitched (2005),Comedy|Fantasy|Romance
"Mexican, The (2001)",Action|Comedy,Rush Hour 2 (2001),Action|Comedy
15 Minutes (2001),Thriller,Panic Room (2002),Thriller
Enemy at the Gates (2001),Drama,Seabiscuit (2003),Drama
Heartbreakers (2001),Comedy|Crime|Romance,Down to Earth (2001),Comedy|Crime|Romance
Spy Kids (2001),Action|Adventure|Children|Comedy,Pirates of the Caribbean: On Stranger Tides (2011),Action|Adventure|Children|Comedy
